In [ ]:
import os
os.environ["QT_QPA_PLATFORM"] = "offscreen"
from qgis.core import QgsApplication
app = QgsApplication([], False)
app.setPrefixPath("/apps/anvil/external/apps/qgis/3.40.1-Bratislava", True)
# Use the actual plugin path
plugin_path = os.path.expanduser("~/.local/share/QGIS/QGIS3/profiles/default/python/plugins")
app.setPluginPath(plugin_path)
app.initQgis()

In [ ]:
import os
import sys
if plugin_path not in sys.path:
	sys.path.append(plugin_path)

import processing
from processing.core.Processing import Processing

try:
    from processing_umep.processing_umep_provider import ProcessingUMEPProvider
    umep_provider = ProcessingUMEPProvider()
    QgsApplication.processingRegistry().addProvider(umep_provider)
    print("UMEP imported")
except Exception as e:
	print("UMEP import error:", e)

In [ ]:
import os
import glob
from osgeo import gdal, osr # osr is needed for SpatialReference
import zipfile
import time
import traceback
from qgis import processing # Assuming QGIS environment is correctly set up
import numpy as np
import multiprocessing
import shutil
# === CONFIGURATION (Identical to your previous script) ===
tile_size = 500
buffer_pixels = 50

# Temporary directories
base_source_data_dir = "/storage/scratch1/4/hyu483/no_heat/UTCI/Input/Final_Run"
temp_base_dir = "/storage/scratch1/4/hyu483/no_heat/UTCI/Temp/Final_Run_208_1300D"
base_tile_buffered_input_dir = os.path.join(temp_base_dir, "Buffered_Inputs")
base_utci_buffered_output_dir = os.path.join(temp_base_dir, "Buffered_UTCI_Output")

PATHS_CONFIG = {
    "TMRT": "/storage/home/hcoda1/4/hyu483/scratch/no_heat/SOLWEIG/Output/Final_Run_2/Interpolated_Output_Scipy/Tmrt_2023_208_1300D.tif",
    "UROCK": os.path.join(base_source_data_dir, "merged_urock_final_filled_scipy.tif"),
    "BUILDING": os.path.join(base_source_data_dir, "buildings.tif"),
    "MET": os.path.join(base_source_data_dir, "metforcing.txt")
}

# Output directories
final_output_base_dir = "/storage/scratch1/4/hyu483/no_heat/UTCI/Output/Final_Run_208_1300D"
base_tile_debuffered_output_dir = os.path.join(final_output_base_dir, "Debuffered_Tiles")
merged_output_dir = os.path.join(final_output_base_dir, "Merged_Output")



In [ ]:
gdal.UseExceptions() # Enable GDAL exceptions

# === HELPER FUNCTION ===
def calculate_gdal_sub_geotransform(parent_gt, x_offset_pixels, y_offset_pixels):
    """
    Calculates the GeoTransform for a sub-window of a raster based on pixel offsets.
    """
    # parent_gt = [ulx, x_res, x_skew, uly, y_skew, y_res]
    new_ulx = parent_gt[0] + x_offset_pixels * parent_gt[1] + y_offset_pixels * parent_gt[2]
    new_uly = parent_gt[3] + x_offset_pixels * parent_gt[4] + y_offset_pixels * parent_gt[5]
    return (new_ulx, parent_gt[1], parent_gt[2], new_uly, parent_gt[4], parent_gt[5])


def gdal_slice_raster_by_window(
    raster_path: str,
    out_path: str,
    slice_c_off: int,  # Column offset (x-pixel offset)
    slice_r_off: int,  # Row offset (y-pixel offset)
    slice_win_width: int, # Width of the window in pixels
    slice_win_height: int, # Height of the window in pixels
    output_format: str = "GTiff",
    resample_alg: str = "nearest", # Resampling algorithm (e.g., 'nearest', 'bilinear').
                                    # Note: For simple cropping with srcWin, this primarily affects
                                    # output if width/height/resolution parameters are also used to
                                    # change the output dimensions from the source window size.
    creation_options: list = None # List of creation options for the output format (e.g., ["TILED=YES", "COMPRESS=LZW"])
):
    """
    Slices a raster image by a pixel window using GDAL's gdal.Translate function.

    Args:
        raster_path (str): Path to the input raster file.
        out_path (str): Path for the output sliced raster file.
        slice_c_off (int): Column offset (x-pixel offset) for the top-left corner of the window.
        slice_r_off (int): Row offset (y-pixel offset) for the top-left corner of the window.
        slice_win_width (int): Width of the window in pixels.
        slice_win_height (int): Height of the window in pixels.
        output_format (str, optional): Output raster format (e.g., "GTiff"). Defaults to "GTiff".
        resample_alg (str, optional): Resampling algorithm. Defaults to "bilinear".
        creation_options (list, optional): List of creation options for the output driver.
                                          Defaults to ["TILED=YES", "COMPRESS=LZW"].

    Returns:
        str: The path to the created output raster file.

    Raises:
        RuntimeError: If GDAL fails to open the input raster or fails during translation.
    """
    if creation_options is None:
        creation_options = ["TILED=YES", "COMPRESS=LZW"]

    # Ensure the output directory exists
    out_dir = os.path.dirname(out_path)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    # Build the TranslateOptions.
    # srcWin parameter is used to specify the window of pixels to extract.
    # It takes a list/tuple: [xoff, yoff, xsize, ysize]
    translate_opts = gdal.TranslateOptions(
        format=output_format,
        srcWin=[slice_c_off, slice_r_off, slice_win_width, slice_win_height],
        resampleAlg=resample_alg,
        creationOptions=creation_options
    )

    # Perform the translation (cropping)
    # gdal.Translate returns a Dataset object on success, or None on failure.
    ds = gdal.Translate(
        destName=out_path,  # Path for the output raster
        srcDS=raster_path,  # Path to the input raster
        options=translate_opts # Translation options
    )

    # Check if the translation was successful
    if ds is None:
        raise RuntimeError("gdal.Translate failed to slice the raster.")

    # Close the dataset explicitly to ensure data is written to disk and resources are released.
    ds = None

    return out_path


# === FUNCTION TO DEBUFFER AND SAVE RASTER (GDAL) ===
def debuffer_and_save_raster_gdal(
    buffered_raster_path: str,
    debuffered_raster_path: str,
    original_full_raster_geotransform: tuple, # GeoTransform of the *original* large source raster
    original_core_tile_col_off_in_full: int, # Column offset of the core tile within the full raster
    original_core_tile_row_off_in_full: int, # Row offset of the core tile within the full raster
    core_tile_width: int,                   # Width of the non-buffered core data
    core_tile_height: int,                  # Height of the non-buffered core data
    actual_buffer_on_left_pixels: int,      # Buffer pixels on the left *within the buffered_raster_path*
    actual_buffer_on_top_pixels: int        # Buffer pixels on the top *within the buffered_raster_path*
    ):
    """
    Clips the core data from a buffered raster using GDAL and saves it
    with correct global georeferencing.
    """
    src_buffered_ds = gdal.Open(buffered_raster_path, gdal.GA_ReadOnly)
    if src_buffered_ds is None:
        print(f"ERROR: Could not open buffered raster: {buffered_raster_path}")
        return None

    try:
        src_buffered_band = src_buffered_ds.GetRasterBand(1)
        if src_buffered_band is None:
            print(f"ERROR: Could not get band from {buffered_raster_path}")
            src_buffered_ds = None
            return None

        # Read the core data from the buffered raster
        # xoff, yoff, xsize, ysize for ReadAsArray
        core_data = src_buffered_band.ReadAsArray(
            xoff=actual_buffer_on_left_pixels,
            yoff=actual_buffer_on_top_pixels,
            win_xsize=core_tile_width,
            win_ysize=core_tile_height
        ).astype(float)

        if core_data is None:
            print(f"ERROR: Failed to read core data from {buffered_raster_path}")
            src_buffered_ds = None
            return None
        
        if core_data.shape[0] != core_tile_height or core_data.shape[1] != core_tile_width:
            print(f"ERROR: Read core data shape ({core_data.shape}) does not match expected ({core_tile_height}, {core_tile_width}) for {debuffered_raster_path}")
            src_buffered_ds = None
            return None


        # Calculate the correct geotransform for this debuffered (core) tile.
        # This transform places the core tile in its correct global position.
        final_core_geotransform = calculate_gdal_sub_geotransform(
            original_full_raster_geotransform,
            original_core_tile_col_off_in_full,
            original_core_tile_row_off_in_full
        )

        # Create the output debuffered raster
        driver = gdal.GetDriverByName("GTiff")
        if driver is None:
            print("ERROR: GTiff driver not available.")
            src_buffered_ds = None
            return None
            
        os.makedirs(os.path.dirname(debuffered_raster_path), exist_ok=True)
        
        # Get data type from source buffered band
        gdal_data_type = src_buffered_band.DataType
        
        dst_ds = driver.Create(
            debuffered_raster_path,
            xsize=core_tile_width,
            ysize=core_tile_height,
            bands=1, # Assuming single band TMRT
            eType=gdal_data_type,
            options=["COMPRESS=LZW"] # Add other options if needed
        )
        if dst_ds is None:
            print(f"ERROR: Could not create output raster: {debuffered_raster_path}")
            src_buffered_ds = None
            return None

        dst_ds.SetGeoTransform(final_core_geotransform)
        dst_ds.SetProjection(src_buffered_ds.GetProjection()) # Preserve CRS

        dst_band = dst_ds.GetRasterBand(1)
        dst_band.WriteArray(core_data)
        no_data_value = src_buffered_band.GetNoDataValue()
        if no_data_value is not None:
            dst_band.SetNoDataValue(no_data_value)
        
        dst_band.FlushCache()
        dst_ds = None # Close and save

        return debuffered_raster_path

    except Exception as e:
        print(f"ERROR during debuffering for {buffered_raster_path} to {debuffered_raster_path}: {e}\n{traceback.format_exc()}")
        return None
    finally:
        if src_buffered_ds:
            src_buffered_ds = None


# === WORKER FUNCTION FOR PROCESSING A SINGLE TILE ===
def process_tile_with_buffer(
    core_r_offset, core_c_offset,
    core_tile_width, core_tile_height,
    buffer_px,
    full_raster_total_width, full_raster_total_height,
    original_full_raster_geotransform_tuple, original_full_raster_crs_wkt, # GDAL specific
    paths_cfg_dict,
    base_buffered_input_dir_worker,
    base_solweig_out_dir_worker,
    base_debuffered_out_dir_worker,
    tile_id_str
    ):
    print(f"[{tile_id_str}] Starting processing with buffer...")
    tile_processing_start_time = time.time()
    debuffered_output_files_for_this_tile = []

    try:
        # --- 1. Calculate Buffered Window for Slicing Inputs ---
        slice_r_off = max(0, core_r_offset - buffer_px)
        slice_c_off = max(0, core_c_offset - buffer_px)
        slice_r_end = min(full_raster_total_height, core_r_offset + core_tile_height + buffer_px)
        slice_c_end = min(full_raster_total_width, core_c_offset + core_tile_width + buffer_px)
        slice_win_height = (slice_r_end - slice_r_off)
        slice_win_width = (slice_c_end - slice_c_off)

        if slice_win_width <= 0 or slice_win_height <= 0:
            msg = f"[{tile_id_str}] Skipped: Calculated buffered slice window has zero/negative dimension. W:{slice_win_width} H:{slice_win_height}"
            print(msg)
            return msg # Return error/skip message

        actual_buffer_left = core_c_offset - slice_c_off
        actual_buffer_top = core_r_offset - slice_r_off

        # --- 2. Prepare Directories for this Tile ---
        current_tile_buffered_data_dir = os.path.join(base_buffered_input_dir_worker, tile_id_str)
        current_tile_utci_output_dir = os.path.join(base_solweig_out_dir_worker, tile_id_str)
        os.makedirs(current_tile_utci_output_dir, exist_ok=True)
        current_tile_utci_output = os.path.join(current_tile_utci_output_dir, "UTCI.tif")
        current_tile_debuffered_output_dir = os.path.join(base_debuffered_out_dir_worker, tile_id_str)
        os.makedirs(current_tile_debuffered_output_dir, exist_ok=True)

        # *** EDITED SECTION 1: CHECK FOR EXISTING OUTPUT ***
        all_files_exist = os.path.exists(current_tile_utci_output)
        if all_files_exist:
            print(f"[{tile_id_str}] Found all required UTCI output files. Skipping processing run.")
        else:
            # --- 3. Slice Main Input Rasters (Buffered) ---
            tile_specific_buffered_inputs = {}
            for key in ["TMRT", "UROCK", "BUILDING"]:
                in_path = paths_cfg_dict[key]
                out_filename = os.path.basename(in_path)
                out_path = os.path.join(current_tile_buffered_data_dir, out_filename)
                tile_specific_buffered_inputs[key] = gdal_slice_raster_by_window(
                    raster_path=in_path, out_path=out_path,
                    slice_c_off=slice_c_off, slice_r_off=slice_r_off,
                    slice_win_width=slice_win_width, slice_win_height=slice_win_height
                )
                if tile_specific_buffered_inputs[key] is None: # Check if slicing failed
                     raise RuntimeError(f"Failed to slice {key} for {tile_id_str}")
                    
            for key in ["MET"]:
                in_path = paths_cfg_dict[key]
                out_filename = os.path.basename(in_path)
                out_path = os.path.join(current_tile_buffered_data_dir, out_filename)
                try:
                    shutil.copy2(in_path, out_path) # copy2 preserves metadata
                    tile_specific_buffered_inputs[key] = out_path
                except Exception as e:
                    raise RuntimeError(f"Failed to copy {key} for {tile_id_str}: {e}")
    
            # --- 5. Run OUTDOOR THERMAL COMFORT SPATIAL THERMAL COMFORT on Buffered Inputs ---
            processing.run("umep:Outdoor Thermal Comfort: Spatial Thermal Comfort", 
               {'TMRT_MAP':tile_specific_buffered_inputs.get('TMRT'),
                'UROCK_MAP':tile_specific_buffered_inputs.get('UROCK'),
                'TC_TYPE':1,'AGE':35,'ACTIVITY':80,'CLO':0.9,'WEIGHT':75,
                'HEIGHT':180,'SEX':0,'COMFA':False,'TC_OUT':current_tile_utci_output})
    
        # *** EDITED SECTION 2: CLEANUP ***
        print(f"[{tile_id_str}] Cleaning up temporary buffered input directory: {current_tile_buffered_data_dir}")
        try:
            shutil.rmtree(current_tile_buffered_data_dir, ignore_errors=True)
        except Exception as e:
            print(f"[{tile_id_str}] WARNING: Could not clean up temporary directory {current_tile_buffered_data_dir}: {e}")

        proc_time = (time.time() - tile_processing_start_time) / 60
        print(f"[{tile_id_str}] Tile task finished in {proc_time:.2f} mins.")
        
        # --- 6. Debuffer the Output ---
        utci_glob_pattern = os.path.join(current_tile_utci_output_dir, "UTCI.tif")
        all_buffered_utci_files = glob.glob(utci_glob_pattern)

        if not all_buffered_utci_files:
            print(f"[{tile_id_str}] WARNING: No TMRT output files found in {current_tile_utci_output_dir} after SOLWEIG run.")
            # This might not be an error if SOLWEIG was not expected to produce TMRT in some cases
            # but usually it's an indication of a problem with the SOLWEIG run itself.
        
        for buffered_utci_path in all_buffered_utci_files:
            buffered_utci_filename_base = os.path.basename(buffered_utci_path)
            debuffered_utci_filename = f"{os.path.splitext(buffered_utci_filename_base)[0]}_{tile_id_str}_debuffered.tif"
            debuffered_utci_path = os.path.join(current_tile_debuffered_output_dir, debuffered_utci_filename)

            if os.path.exists(debuffered_utci_path):
                debuffered_output_files_for_this_tile.append(debuffered_utci_path)
                print(f"[{tile_id_str}] already exists: {debuffered_utci_path}")
            else:
                # Parameters for debuffer_and_save_raster_gdal:
                debuffered_file = debuffer_and_save_raster_gdal(
                    buffered_raster_path=buffered_utci_path,
                    debuffered_raster_path=debuffered_utci_path,
                    original_full_raster_geotransform=original_full_raster_geotransform_tuple,
                    original_core_tile_col_off_in_full=core_c_offset,
                    original_core_tile_row_off_in_full=core_r_offset,
                    core_tile_width=core_tile_width,
                    core_tile_height=core_tile_height,
                    actual_buffer_on_left_pixels=actual_buffer_left,
                    actual_buffer_on_top_pixels=actual_buffer_top
                )
                if debuffered_file:
                    debuffered_output_files_for_this_tile.append(debuffered_file)
                    print(f"[{tile_id_str}] Successfully debuffered: {debuffered_file}")
                else:
                    print(f"[{tile_id_str}] ERROR failed to debuffer: {buffered_tmrt_path}")

        if not debuffered_output_files_for_this_tile and all_buffered_utci_files:
            msg = f"[{tile_id_str}] ERROR: Found TMRT files but failed to debuffer any."
            print(msg)
            return msg
        
        proc_time = (time.time() - tile_processing_start_time) / 60
        msg = f"[{tile_id_str}] Tile processing (buffered) complete in {proc_time:.2f} mins. {len(debuffered_output_files_for_this_tile)} TMRT files debuffered."
        print(msg)
        return debuffered_output_files_for_this_tile

    except Exception as e:
        err_msg = f"[{tile_id_str}] ERROR processing tile: {e}\n{traceback.format_exc()}"
        print(err_msg)
        return err_msg


# === MAIN SCRIPT EXECUTION ===
def main():
    overall_start_time = time.time()
    os.makedirs(base_tile_buffered_input_dir, exist_ok=True)
    os.makedirs(base_utci_buffered_output_dir, exist_ok=True)
    os.makedirs(base_tile_debuffered_output_dir, exist_ok=True)
    os.makedirs(merged_output_dir, exist_ok=True)

    # --- Get dimensions and georeferencing from one of the main full-sized rasters using GDAL ---
    ref_ds = gdal.Open(PATHS_CONFIG['TMRT'], gdal.GA_ReadOnly)
    if ref_ds is None:
        print(f"CRITICAL ERROR: Could not open reference raster: {PATHS_CONFIG['TMRT']}")
        return
    full_raster_width = ref_ds.RasterXSize
    full_raster_height = ref_ds.RasterYSize
    original_raster_geotransform = ref_ds.GetGeoTransform() # Tuple
    original_raster_crs_wkt = ref_ds.GetProjection()     # WKT string
    ref_ds = None # Close it
    print(f"Full source raster dimensions: {full_raster_width}x{full_raster_height}, Tile Size: {tile_size}x{tile_size}, Buffer: {buffer_pixels}px")
    print(f"Source GeoTransform: {original_raster_geotransform}")
    print(f"Source CRS (WKT): {original_raster_crs_wkt[:100]}...") # Print first 100 chars

    tasks_for_pool = []

    for r_idx, core_r_off in enumerate(range(0, full_raster_height, tile_size)):
        for c_idx, core_c_off in enumerate(range(0, full_raster_height, tile_size)):
            current_core_tile_width = min(tile_size, full_raster_width - core_c_off)
            current_core_tile_height = min(tile_size, full_raster_height - core_r_off)

            if current_core_tile_width <= 0 or current_core_tile_height <= 0:
                print(f"Skipping task generation for zero-dimension core tile at r_offset={core_r_off}, c_offset={core_c_off}")
                continue
            
            tile_identifier_string = f"tile_{r_idx}_{c_idx}"
            task_args = (
                core_r_off, core_c_off,
                current_core_tile_width, current_core_tile_height,
                buffer_pixels,
                full_raster_width, full_raster_height,
                original_raster_geotransform, original_raster_crs_wkt, # Pass GDAL specific info
                PATHS_CONFIG,
                base_tile_buffered_input_dir,
                base_utci_buffered_output_dir,
                base_tile_debuffered_output_dir,
                tile_identifier_string
            )
            tasks_for_pool.append(task_args)

    if not tasks_for_pool:
        print("No tasks generated.")
        return

    print(f"\nGenerated {len(tasks_for_pool)} tasks for multiprocessing.")
    num_processes = min(max(1, os.cpu_count() - 2), 40)
    # num_processes = 1 # For testing
    print(f"Using {num_processes} processes.")

    with multiprocessing.Pool(processes=num_processes) as pool:
        results_from_pool = pool.starmap(process_tile_with_buffer, tasks_for_pool)
    
    print("\n=== Multiprocessing of tiles complete. ===")

    organized_debuffered_files = {}
    failed_tile_processing_count = 0
    for result_item in results_from_pool:
        if isinstance(result_item, str) and ("ERROR" in result_item or "Skipped" in result_item or "WARNING" in result_item): # Check for string error messages
            failed_tile_processing_count += 1
            # Error/skip message already printed by worker
        elif isinstance(result_item, list): # Successful result is a list of debuffered paths
            for debuffered_path in result_item:
                if os.path.exists(debuffered_path):
                    filename = os.path.basename(debuffered_path)
                    parts = filename.split('_tile_')[0]
                    tmrt_type_key = parts
                    if tmrt_type_key not in organized_debuffered_files:
                        organized_debuffered_files[tmrt_type_key] = []
                    organized_debuffered_files[tmrt_type_key].append(debuffered_path)
                else:
                    print(f"Warning: Worker reported debuffered file, but not found: {debuffered_path}")
        else: # Unrecognized result type
            failed_tile_processing_count +=1
            print(f"  Unknown result type from worker for a tile: {type(result_item)} - {str(result_item)[:200]}")


    print(f"\nTile processing summary: {len(results_from_pool) - failed_tile_processing_count} tiles had successful outcomes (returned list of paths).")
    print(f"Tiles that returned error/skip messages or unknown result types: {failed_tile_processing_count}")


    if not organized_debuffered_files:
        print("No debuffered files were successfully organized for merging.")
        overall_proc_time_early_exit = (time.time() - overall_start_time) / 60
        print(f"\nTotal script execution time: {overall_proc_time_early_exit:.2f} minutes.")
        return

    # --- Merge Each Set of Debuffered TMRT Tiles using GDAL ---
    for tmrt_type, file_list_to_merge in organized_debuffered_files.items():
        if not file_list_to_merge:
            print(f"No files to merge for TMRT type: {tmrt_type}")
            continue

        print(f"\nAttempting to merge {len(file_list_to_merge)} debuffered tiles for TMRT type: {tmrt_type}...")
        merged_tmrt_output_path = os.path.join(merged_output_dir, f"{tmrt_type}_merged_final.tif")
        vrt_path = os.path.join(merged_output_dir, f"{tmrt_type}_temp.vrt") # Temporary VRT

        try:
            # 1. Build VRT
            # Ensure file_list_to_merge contains valid paths
            valid_files_for_vrt = [f for f in file_list_to_merge if os.path.exists(f)]
            if not valid_files_for_vrt:
                print(f"ERROR: No valid source files found for merging {tmrt_type} after checking existence.")
                continue
            
            vrt_options = gdal.BuildVRTOptions(resampleAlg='nearest', addAlpha=False) # Adjust options as needed
            vrt_ds = gdal.BuildVRT(vrt_path, valid_files_for_vrt, options=vrt_options)
            if vrt_ds is None:
                print(f"ERROR: Failed to build VRT for {tmrt_type}.")
                continue
            vrt_ds = None # Close VRT dataset

            # 2. Translate VRT to final TIFF
            nodata_val = None
            if valid_files_for_vrt:
                first_tile_ds = gdal.Open(valid_files_for_vrt[0])
                if first_tile_ds:
                    nodata_val = first_tile_ds.GetRasterBand(1).GetNoDataValue()
                    first_tile_ds = None # Close it

            translate_options_list = ["COMPRESS=LZW", "TILED=YES", "BIGTIFF=IF_SAFER"]
            if nodata_val is not None:
                # gdal.Translate expects NoData as a string if it's part of creationOptions,
                # or use the noData parameter directly.
                # For gdal.Translate, the parameter is `noData`
                final_ds = gdal.Translate(merged_tmrt_output_path, vrt_path,
                                          format="GTiff",
                                          creationOptions=translate_options_list,
                                          noData=nodata_val if nodata_val is not None else 'none') # Use 'none' if no nodata
            else:
                 final_ds = gdal.Translate(merged_tmrt_output_path, vrt_path,
                                          format="GTiff",
                                          creationOptions=translate_options_list)


            if final_ds is None:
                print(f"ERROR: Failed to translate VRT to TIFF for {tmrt_type}.")
                if os.path.exists(vrt_path): os.remove(vrt_path) # Clean up VRT
                continue
            final_ds = None # Close final dataset

            print(f"Successfully merged {tmrt_type} tiles to: {merged_tmrt_output_path}")
            if os.path.exists(vrt_path):
                os.remove(vrt_path) # Clean up VRT

        except Exception as e:
            print(f"ERROR during GDAL merging of {tmrt_type} tiles: {e}\n{traceback.format_exc()}")
            if os.path.exists(vrt_path) and os.path.isfile(vrt_path): # ensure it's a file before removing
                 try:
                     os.remove(vrt_path)
                 except OSError as ose:
                     print(f"Could not remove temporary VRT {vrt_path}: {ose}")


    overall_proc_time = (time.time() - overall_start_time) / 60
    print(f"\nTotal script execution time: {overall_proc_time:.2f} minutes.")

In [ ]:
if __name__ == '__main__':
    main()